In [1]:
# 1,加载数据：
import pandas as pd

data = pd.read_excel('../DataSets/test_data.xlsx')

print(data.shape, '\n', data.dtypes)
data.head()

(3000, 6) 
 id              int64
gender         object
age           float64
edu            object
custom_amt     object
order_date     object
dtype: object


,id,gender,age,edu,custom_amt,order_date
0,890,female,43.0,NaN,￥2177.94,2018年12月25日
1,2391,male,52.0,NaN,￥2442.18,2017年5月24日
2,2785,male,39.0,NaN,￥849.79,2018年5月15日
3,1361,female,26.0,NaN,￥2482.22,2018年5月16日
4,888,female,61.0,本科,￥2027.9,2018年1月21日


In [2]:
# 2,属性类型转换：
data['id'] = data['id'].astype(str)  # 数值型 转 字符型

data['custom_amt'] = data['custom_amt'].str[1:].astype(float)  # 字符型转数值型：去调dollar符，规范化数据

data['order_date'] = pd.to_datetime(data['order_date'], format = '%Y年%m月%d日')   # 字符型 转 日期型
data.dtypes

id                    object
gender                object
age                  float64
edu                   object
custom_amt           float64
order_date    datetime64[ns]
dtype: object

In [3]:
# 3，判断数据中是否存在重复观测
print(data.duplicated().any())

df = pd.DataFrame(dict(name = ['张三','李四','王二','张三','赵五','丁一','王二'],
                      gender = ['男','男','女','男','女','女','男'],
                      age = [29,25,27,29,21,22,27],
                      income = [15600,14000,18500,15600,10500,18000,13000],
                      edu = ['本科','本科','硕士','本科','大专','本科','硕士']))
print(df)
print(df.duplicated().any())
df.drop_duplicates()   # 默认情况下，对数据的所有变量进行判断
df.drop_duplicates(subset=['name','age'], inplace=True)   # inplace=True:改变原有数据,默认为False.
df

False
  name gender  age  income edu
0   张三      男   29   15600  本科
1   李四      男   25   14000  本科
2   王二      女   27   18500  硕士
3   张三      男   29   15600  本科
4   赵五      女   21   10500  大专
5   丁一      女   22   18000  本科
6   王二      男   27   13000  硕士
True


,name,gender,age,income,edu
0,张三,男,29,15600,本科
1,李四,男,25,14000,本科
2,王二,女,27,18500,硕士
4,赵五,女,21,10500,大专
5,丁一,女,22,18000,本科


In [4]:
# 4,缺省值判断：
data.isnull().any(axis = 0)                 # 判断各变量（列）中是否存在缺失值

data.isnull().sum(axis = 0)                 # 各变量中缺失值的数量

res1 = data.isnull().sum(axis = 0)/data.shape[0]   # 各变量中缺失值的比例


data.isnull().any(axis = 1).any()          # 判断数据行中是否存在缺失值

data.isnull().any(axis = 1).sum()          # 缺失观测的行数

res2 = data.isnull().any(axis = 1).sum()/data.shape[0] # 缺失观测的比例
print(res1, '\n***\n',res2)

id            0.000000
gender        0.045333
age           0.033333
edu           0.642333
custom_amt    0.000000
order_date    0.000000
dtype: float64 
***
 0.6746666666666666


In [5]:
# 5，缺省值处理：
data.drop(labels='edu', axis = 1, inplace=True)   # 删除字段： 删除缺失率非常高的edu变量，即删除列

data_new = data.drop(labels=data.index[data['age'].isnull()], axis = 0)  # 删除age变量中所对应的缺失观测样本

# 替换法处理缺失值
data.fillna(value = {'gender': data['gender'].mode()[0],   # 众数替换缺失值
                 'age':data['age'].mean()                   # 平均值替换缺失值
                 },
          inplace = True                                     # 原地修改原始数据
          )

data.isnull().sum(axis = 0)                                 # 再次查看各变量的缺失比例

id            0
gender        0
age           0
custom_amt    0
order_date    0
dtype: int64

In [8]:
# 6，Titanic预测项目实践：

# 6.1，加载数据，缺省值处理：
titanic = pd.read_csv('../DataSets/Titanic/train.csv')

titanic.drop(labels='Cabin', axis = 1, inplace=True)  # 删除缺失严重的Cabin变量，即删除列

titanic.dropna(subset=['Embarked'], inplace=True)   # 根据Embarked变量，删除对应的缺失行

titanic.drop(labels=['PassengerId','Name','Ticket','Embarked'], axis = 1, inplace=True)  # 删除无关紧要的变量（这些变量对后面预测年龄没有太多的帮助）

titanic.Sex = titanic.Sex.map({'male':1, 'female':0})  # 将字符型的性别变量映射为数值变量


no_missing = titanic.loc[~titanic.Age.isnull(),]# 将数据拆分为两组，一是年龄缺失组，二是年龄非缺失组，后续基于非缺失值构建KNN模型，再对缺失组做预测
missing = titanic.loc[titanic.Age.isnull(),]
print(no_missing.shape, missing.shape)

from sklearn import neighbors

X = nomissing.columns[nomissing.columns != 'Age']   # 自变量

knn = neighbors.KNeighborsRegressor()  # 构建模型

knn.fit(nomissing[X], nomissing.Age)   # 模型拟合

pre_age = knn.predict(missing[X])      # 预测
pre_age

(712, 7) (177, 7)


array([28.4  , 33.2  , 18.2  , 32.8  , 24.   , 33.   , 21.584, 19.6  ,
       24.4  , 33.   , 30.3  , 30.   , 19.6  , 26.4  , 49.4  , 47.3  ,
       14.534, 33.   , 30.3  , 19.6  , 30.3  , 30.3  , 33.   , 27.6  ,
       31.4  , 30.3  , 49.   , 18.2  , 26.7  , 34.8  , 30.   , 37.   ,
       33.2  , 56.6  , 31.8  , 37.   , 32.8  , 45.4  , 28.8  , 49.   ,
       19.6  , 37.   , 23.8  , 33.   , 29.6  , 27.9  , 25.9  , 28.8  ,
       34.8  , 35.4  , 49.   , 27.9  , 41.4  , 19.6  , 36.6  , 56.6  ,
       47.3  , 43.4  , 19.6  , 24.4  , 29.8  , 30.3  , 29.   , 37.   ,
       23.2  , 40.4  , 33.   , 24.   , 49.4  , 32.8  , 24.   , 24.   ,
       30.   , 18.2  , 19.6  , 36.4  , 33.   , 39.9  , 29.6  , 33.   ,
       28.8  , 36.6  , 22.8  , 33.   , 34.8  , 49.   , 24.   , 26.   ,
       26.   , 30.3  , 45.   , 49.   , 30.3  , 36.6  , 39.9  , 34.8  ,
       44.4  , 36.6  , 29.6  , 26.   , 28.6  , 27.7  , 27.9  , 41.4  ,
       30.3  , 32.6  , 32.8  , 26.8  , 30.8  , 26.8  , 22.8  , 28.6  ,
      